<a href="https://colab.research.google.com/github/RomanaNourin/Skin-Cancer/blob/main/New__GAN_aurorad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Training GAN

In [ ]:
import pandas as pd
data_dir = '/content/drive/MyDrive/Skin HAM10000/Latest/HAM10000_metadata.csv'
data = pd.read_csv(data_dir)
data

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear
...,...,...,...,...,...,...,...
10010,HAM_0002867,ISIC_0033084,akiec,histo,40.0,male,abdomen
10011,HAM_0002867,ISIC_0033550,akiec,histo,40.0,male,abdomen
10012,HAM_0002867,ISIC_0033536,akiec,histo,40.0,male,abdomen
10013,HAM_0000239,ISIC_0032854,akiec,histo,80.0,male,face


In [ ]:
# Filter out rows where 'dx' == 'nv'
filtered_data = data[data['dx'] != 'nv']

# Select only 'image_id' and 'dx' columns
minority_data = filtered_data[['image_id', 'dx']]

In [ ]:
print(minority_data)

           image_id     dx
0      ISIC_0027419    bkl
1      ISIC_0025030    bkl
2      ISIC_0026769    bkl
3      ISIC_0025661    bkl
4      ISIC_0031633    bkl
...             ...    ...
10010  ISIC_0033084  akiec
10011  ISIC_0033550  akiec
10012  ISIC_0033536  akiec
10013  ISIC_0032854  akiec
10014  ISIC_0032258    mel

[3310 rows x 2 columns]


In [ ]:
image_base_path = '/content/drive/MyDrive/Skin HAM10000/HAM10000_images/'
minority_data['image_path'] = image_base_path + minority_data['image_id'] + '.jpg'
# Save to a new CSV file
minority_data.to_csv('/content/drive/MyDrive/Skin HAM10000/Latest/minority_train.csv', index=False)

# Display the first few rows of the minority dataset
minority_data.head()

<ipython-input-5-0ef65282b3a6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  minority_data['image_path'] = image_base_path + minority_data['image_id'] + '.jpg'


,image_id,dx,image_path
0,ISIC_0027419,bkl,/content/drive/MyDrive/Skin HAM10000/HAM10000_...
1,ISIC_0025030,bkl,/content/drive/MyDrive/Skin HAM10000/HAM10000_...
2,ISIC_0026769,bkl,/content/drive/MyDrive/Skin HAM10000/HAM10000_...
3,ISIC_0025661,bkl,/content/drive/MyDrive/Skin HAM10000/HAM10000_...
4,ISIC_0031633,bkl,/content/drive/MyDrive/Skin HAM10000/HAM10000_...


In [ ]:
# prompt: show head minority_train.csv size

import pandas as pd

# Load the CSV file
minority_data = pd.read_csv('/content/drive/MyDrive/Skin HAM10000/Latest/minority_train.csv')

# Display the size of the DataFrame
print(minority_data.shape)

# Display the first few rows
minority_data.head()

(3310, 3)


,image_id,dx,image_path
0,ISIC_0027419,bkl,/content/drive/MyDrive/Skin HAM10000/HAM10000_...
1,ISIC_0025030,bkl,/content/drive/MyDrive/Skin HAM10000/HAM10000_...
2,ISIC_0026769,bkl,/content/drive/MyDrive/Skin HAM10000/HAM10000_...
3,ISIC_0025661,bkl,/content/drive/MyDrive/Skin HAM10000/HAM10000_...
4,ISIC_0031633,bkl,/content/drive/MyDrive/Skin HAM10000/HAM10000_...


In [ ]:
from os import path
import sys
sys.path.append(path.abspath('../src/'))

import argparse
import os
import numpy as np
import math

import torchvision.transforms as transforms
from torchvision.utils import save_image
import torch.nn.utils.spectral_norm as spectral_norm


from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

import matplotlib.pyplot as plt

#disable interactive plotting
plt.ioff()

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from torch.utils.data import Dataset, DataLoader, Subset, random_split, SubsetRandomSampler, ConcatDataset

from PIL import Image

class SkinCancer(Dataset):
    """Skin Cancer Dataset."""

    def __init__(self, root_dir, meta, transform=None):
        """
        Args:
            root_dir (string): Path to root directory containing images
            meta_file (string): Path to csv file containing images metadata (image_id, class)

            transform (callable, optional): Optional transform to be applied
            on a sample.
        """

        self.root_dir = root_dir
        self.meta = meta
        self.transform = transform

        self.df = pd.read_csv(self.meta)
        self.image_paths = self.df['image_path'].to_list()
        self.image_ids = self.df['image_id'].to_list()
        self.classes = sorted(self.df['dx'].unique().tolist())
        self.classes_all = self.df['dx'].tolist()


        self.class_id = {i:j for i, j in enumerate(self.classes)}
        self.class_to_id = {value:key for key,value in self.class_id.items()}

        self.class_count =  self.df['dx'].value_counts().to_dict()
        self.transform = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.df.iloc[idx, -1]
        label = self.df.iloc[idx, 1]
        image = Image.open(img_path)
        image_tensor = self.transform(image)
        label_id = torch.tensor(self.class_to_id[str(label)])
        return image_tensor, label_id


In [ ]:
# Loading data
data_dir = '/content/drive/MyDrive/Skin HAM10000/HAM10000_images'
train_data = SkinCancer(data_dir, '/content/drive/MyDrive/Skin HAM10000/Latest/minority_train.csv', transform=None)
dataset_size = len(train_data)
classes=np.unique(train_data.classes)

dataloader = torch.utils.data.DataLoader(train_data, batch_size=16)


unnormalize =transforms.Normalize((-0.5 / 0.5), (1.0 / 0.5))

In [ ]:
# Example: Applying unnormalize
for images, labels in dataloader:
    images = unnormalize(images)  # Reverse normalization
    plt.imshow(images[0].permute(1, 2, 0).numpy())  # Visualize
    break

In [ ]:
# Check if data in dataloader is correct
i,l = train_data.__getitem__(100)
i.shape

torch.Size([3, 224, 224])

In [ ]:
n_epochs=60
#batch_size = 64
lr=0.0001
b1=0.5
b2=0.999
n_cpu=8
latent_dim=100
n_classes=7
img_size=224
channels=1
sample_interval=400
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.label_emb = nn.Embedding(n_classes, latent_dim)

        self.init_size = img_size // 4  # Initial size before upsampling
        self.l1 = nn.Sequential(nn.Linear(latent_dim, 128 * self.init_size ** 2))

        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 3, 3, stride=1, padding=1),
            nn.Tanh(),
        )

    def forward(self, noise, labels):
        gen_input = torch.mul(self.label_emb(labels), noise)
        out = self.l1(gen_input)
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, bn=True):
            """Returns layers of each discriminator block"""
            block = [spectral_norm(nn.Conv2d(in_filters, out_filters, 3, 2, 1)), nn.LeakyReLU(0.2, inplace=True), nn.Dropout2d(0.25)]
            if bn:
                block.append(nn.BatchNorm2d(out_filters, 0.8))
            return block

        self.conv_blocks = nn.Sequential(
            *discriminator_block(3, 16, bn=False),
            *discriminator_block(16, 32),
            *discriminator_block(32, 64),
            *discriminator_block(64, 128),
        )

        # The height and width of downsampled image
        ds_size = img_size // 2 ** 4

        # Output layers
        self.adv_layer = nn.Sequential(nn.Linear(128 * ds_size ** 2, 1), nn.Sigmoid())
        self.aux_layer = nn.Sequential(nn.Linear(128 * ds_size ** 2, n_classes), nn.Softmax())

    def forward(self, img):
        out = self.conv_blocks(img)
        out = out.view(out.shape[0], -1)
        validity = self.adv_layer(out)
        label = self.aux_layer(out)

        return validity, label

In [ ]:
# Loss functions
adversarial_loss = torch.nn.BCELoss()
auxiliary_loss = torch.nn.CrossEntropyLoss()

# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

if device == 'cuda':
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()
    auxiliary_loss.cuda()

# Initialize weights
generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)

Discriminator(
  (conv_blocks): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Dropout2d(p=0.25, inplace=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Dropout2d(p=0.25, inplace=False)
    (6): BatchNorm2d(32, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (7): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (8): LeakyReLU(negative_slope=0.2, inplace=True)
    (9): Dropout2d(p=0.25, inplace=False)
    (10): BatchNorm2d(64, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (11): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (12): LeakyReLU(negative_slope=0.2, inplace=True)
    (13): Dropout2d(p=0.25, inplace=False)
    (14): BatchNorm2d(128, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
  )
  (ad

In [ ]:
# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=0.0001, betas=(b1, b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=0.0001, betas=(b1, b2))

FloatTensor = torch.cuda.FloatTensor if device == 'cuda' else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if device == 'cuda' else torch.LongTensor

In [ ]:
# Display images
def sample_imgs(gen_imgs, gen_labels, epoch):
    """Saves a grid of generated digits ranging from 0 to n_classes"""
    # Sample noise
    imgs = [i.detach().cpu().squeeze().permute(1, 2, 0) for i in gen_imgs]
    imgs = [unnormalize(i) for i in imgs]
    gen_lab = [int(i.detach().cpu().numpy().tolist()) for i in gen_labels]

    # Check if the generated label is within the range of valid keys in train_data.class_id
    # If not, assign a default title or handle the error appropriately.
    titles = [train_data.class_id.get(i, 'Unknown') for i in gen_lab] # Use .get() with a default value

    f, axes = plt.subplots(4, 4, figsize=(12, 10))

    for idx, img in enumerate(imgs):
        i = idx % 4
        j = idx // 4
        axes[i, j].imshow(img)
        plt.subplots_adjust(wspace=.3, hspace=0.3)

    plt.savefig(f'/content/drive/MyDrive/Skin HAM10000/Latest/gan_save2/epoch_{epoch + 1}_generated_batch.jpg', bbox_inches='tight')

In [ ]:
# ----------
#  Training
# ----------
gen_loss = 100.0
dis_loss = 100.0
device='cuda'
for epoch in range(n_epochs):
    for i, (imgs, labels) in enumerate(dataloader):

        if i<165:

            batch_size = 16
            imgs = imgs.to(device)
            labels = labels.to(device)

            # Adversarial ground truths
            valid = Variable(FloatTensor(batch_size, 1).fill_(1.0), requires_grad=False)
            fake = Variable(FloatTensor(batch_size, 1).fill_(0.0), requires_grad=False)

            # Configure input
            # real_imgs = Variable(imgs.type(FloatTensor))
            # labels = Variable(labels.type(LongTensor))

            real_imgs = imgs
            labels = LongTensor(labels)
            # -----------------
            #  Train Generator
            # -----------------

            optimizer_G.zero_grad()

            # Sample noise and labels as generator input
            z = Variable(FloatTensor(np.random.normal(0, 1, (batch_size, latent_dim))))
            gen_labels = Variable(LongTensor(np.random.randint(0, n_classes, batch_size)))

            # Generate a batch of images
            gen_imgs = generator(z, gen_labels)

            # Loss measures generator's ability to fool the discriminator
            validity, pred_label = discriminator(gen_imgs)
            g_loss = 0.5 * (adversarial_loss(validity, valid) + auxiliary_loss(pred_label, gen_labels))

            g_loss.backward()
            optimizer_G.step()

            # ---------------------
            #  Train Discriminator
            # ---------------------

            optimizer_D.zero_grad()

            # Loss for real images
            real_pred, real_aux = discriminator(real_imgs)
            d_real_loss = (adversarial_loss(real_pred, valid) + auxiliary_loss(real_aux, labels)) / 2

            # Loss for fake images
            fake_pred, fake_aux = discriminator(gen_imgs.detach())
            d_fake_loss = (adversarial_loss(fake_pred, fake) + auxiliary_loss(fake_aux, gen_labels)) / 2

            # Total discriminator loss
            d_loss = (d_real_loss + d_fake_loss) / 2

            # Calculate discriminator accuracy
            pred = np.concatenate([real_aux.data.cpu().numpy(), fake_aux.data.cpu().numpy()], axis=0)
            gt = np.concatenate([labels.data.cpu().numpy(), gen_labels.data.cpu().numpy()], axis=0)
            d_acc = np.mean(np.argmax(pred, axis=1) == gt)

            d_loss.backward()
            optimizer_D.step()

            if g_loss.item() < gen_loss:
                gen_loss = g_loss.item()
                torch.save(generator.state_dict(), f'/content/drive/MyDrive/Skin HAM10000/Latest/gan_models/{generator._get_name()}.pth')


            if d_loss.item() < dis_loss:
                dis_loss = d_loss.item()
                torch.save(discriminator.state_dict(), f'/content/drive/MyDrive/Skin HAM10000/Latest/gan_models/{discriminator._get_name()}.pth')


    sample_imgs(gen_imgs,gen_labels,epoch)
    print(
        "[Epoch %d/%d] [Batch %d/%d] [D loss: %f, acc: %d%%] [G loss: %f]"
        % (epoch, n_epochs, i, len(dataloader), d_loss.item(), 100 * d_acc, g_loss.item())
    )

[Epoch 0/60] [Batch 206/207] [D loss: 1.277109, acc: 25%] [G loss: 1.368407]
[Epoch 1/60] [Batch 206/207] [D loss: 1.207365, acc: 34%] [G loss: 1.278832]
[Epoch 2/60] [Batch 206/207] [D loss: 1.284578, acc: 21%] [G loss: 1.394839]
[Epoch 3/60] [Batch 206/207] [D loss: 1.244674, acc: 37%] [G loss: 1.299111]
[Epoch 4/60] [Batch 206/207] [D loss: 1.271100, acc: 34%] [G loss: 1.277734]
[Epoch 5/60] [Batch 206/207] [D loss: 1.256944, acc: 34%] [G loss: 1.330308]
[Epoch 6/60] [Batch 206/207] [D loss: 1.190638, acc: 37%] [G loss: 1.194115]
[Epoch 7/60] [Batch 206/207] [D loss: 1.268338, acc: 18%] [G loss: 1.380735]
[Epoch 8/60] [Batch 206/207] [D loss: 1.172522, acc: 37%] [G loss: 1.213503]
[Epoch 9/60] [Batch 206/207] [D loss: 1.295730, acc: 28%] [G loss: 1.306083]
[Epoch 10/60] [Batch 206/207] [D loss: 1.167049, acc: 43%] [G loss: 1.307572]
[Epoch 11/60] [Batch 206/207] [D loss: 1.208759, acc: 43%] [G loss: 1.244950]
[Epoch 12/60] [Batch 206/207] [D loss: 1.039801, acc: 62%] [G loss: 1.1636

In [ ]:
# ----------
#  Training
# ----------
gen_loss = 100.0
dis_loss = 100.0
device='cuda'
for epoch in range(n_epochs):
    for i, (imgs, labels) in enumerate(dataloader):

        if i<165:

            batch_size = 16
            imgs = imgs.to(device)
            labels = labels.to(device)

            # Adversarial ground truths
            valid = Variable(FloatTensor(batch_size, 1).fill_(1.0), requires_grad=False)
            fake = Variable(FloatTensor(batch_size, 1).fill_(0.0), requires_grad=False)

            # Configure input
            # real_imgs = Variable(imgs.type(FloatTensor))
            # labels = Variable(labels.type(LongTensor))

            real_imgs = imgs
            labels = LongTensor(labels)
            # -----------------
            #  Train Generator
            # -----------------

            optimizer_G.zero_grad()

            # Sample noise and labels as generator input
            z = Variable(FloatTensor(np.random.normal(0, 1, (batch_size, latent_dim))))
            gen_labels = Variable(LongTensor(np.random.randint(0, n_classes, batch_size)))

            # Generate a batch of images
            gen_imgs = generator(z, gen_labels)

            # Loss measures generator's ability to fool the discriminator
            validity, pred_label = discriminator(gen_imgs)
            g_loss = 0.5 * (adversarial_loss(validity, valid) + auxiliary_loss(pred_label, gen_labels))

            g_loss.backward()
            optimizer_G.step()

            # ---------------------
            #  Train Discriminator
            # ---------------------

            optimizer_D.zero_grad()

            # Loss for real images
            real_pred, real_aux = discriminator(real_imgs)
            d_real_loss = (adversarial_loss(real_pred, valid) + auxiliary_loss(real_aux, labels)) / 2

            # Loss for fake images
            fake_pred, fake_aux = discriminator(gen_imgs.detach())
            d_fake_loss = (adversarial_loss(fake_pred, fake) + auxiliary_loss(fake_aux, gen_labels)) / 2

            # Total discriminator loss
            d_loss = (d_real_loss + d_fake_loss) / 2

            # Calculate discriminator accuracy
            pred = np.concatenate([real_aux.data.cpu().numpy(), fake_aux.data.cpu().numpy()], axis=0)
            gt = np.concatenate([labels.data.cpu().numpy(), gen_labels.data.cpu().numpy()], axis=0)
            d_acc = np.mean(np.argmax(pred, axis=1) == gt)

            d_loss.backward()
            optimizer_D.step()

            if g_loss.item() < gen_loss:
                gen_loss = g_loss.item()
                torch.save(generator.state_dict(), f'/content/drive/MyDrive/Skin HAM10000/Latest/gan_models2/{generator._get_name()}.pth')


            if d_loss.item() < dis_loss:
                dis_loss = d_loss.item()
                torch.save(discriminator.state_dict(), f'/content/drive/MyDrive/Skin HAM10000/Latest/gan_models2/{discriminator._get_name()}.pth')


    sample_imgs(gen_imgs,gen_labels,epoch)
    print(
        "[Epoch %d/%d] [Batch %d/%d] [D loss: %f, acc: %d%%] [G loss: %f]"
        % (epoch, n_epochs, i, len(dataloader), d_loss.item(), 100 * d_acc, g_loss.item())
    )

[Epoch 0/60] [Batch 206/207] [D loss: 0.919609, acc: 81%] [G loss: 1.019026]
[Epoch 1/60] [Batch 206/207] [D loss: 0.998121, acc: 87%] [G loss: 0.828333]
[Epoch 2/60] [Batch 206/207] [D loss: 0.943943, acc: 90%] [G loss: 0.848627]
[Epoch 3/60] [Batch 206/207] [D loss: 0.920650, acc: 84%] [G loss: 0.936049]
[Epoch 4/60] [Batch 206/207] [D loss: 0.892838, acc: 87%] [G loss: 1.055627]
[Epoch 5/60] [Batch 206/207] [D loss: 0.916878, acc: 90%] [G loss: 1.062262]
[Epoch 6/60] [Batch 206/207] [D loss: 0.895530, acc: 93%] [G loss: 1.030620]
[Epoch 7/60] [Batch 206/207] [D loss: 0.959324, acc: 90%] [G loss: 1.078652]
[Epoch 8/60] [Batch 206/207] [D loss: 0.902230, acc: 84%] [G loss: 1.033213]
[Epoch 9/60] [Batch 206/207] [D loss: 0.963904, acc: 90%] [G loss: 0.907964]
[Epoch 10/60] [Batch 206/207] [D loss: 0.937856, acc: 90%] [G loss: 1.101973]
[Epoch 11/60] [Batch 206/207] [D loss: 0.867692, acc: 96%] [G loss: 0.941053]
[Epoch 12/60] [Batch 206/207] [D loss: 0.913656, acc: 96%] [G loss: 0.9847

# Generating synthetic images

In [ ]:
import torch

generator = Generator()
generator.load_state_dict(torch.load('/content/drive/MyDrive/Skin HAM10000/Latest/gan_models2/Generator.pth'))
generator.eval()

Generator(
  (label_emb): Embedding(7, 100)
  (l1): Sequential(
    (0): Linear(in_features=100, out_features=401408, bias=True)
  )
  (conv_blocks): Sequential(
    (0): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Upsample(scale_factor=2.0, mode='nearest')
    (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): BatchNorm2d(128, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Upsample(scale_factor=2.0, mode='nearest')
    (6): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): BatchNorm2d(64, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (8): LeakyReLU(negative_slope=0.2, inplace=True)
    (9): Conv2d(64, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): Tanh()
  )
)

In [ ]:
import pandas as pd

minority_data = pd.read_csv('/content/drive/MyDrive/Skin HAM10000/Latest/minority_train.csv')

class_counts = minority_data['dx'].value_counts()
print(class_counts)

dx
mel      1113
bkl      1099
bcc       514
akiec     327
vasc      142
df        115
Name: count, dtype: int64


In [ ]:
import os
from torchvision.utils import save_image

latent_dim = 100
batch_size = 1  # Use a batch size of 1

output_dir = '/content/drive/MyDrive/Skin HAM10000/Latest/Synthetic_Images/'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
generator.to(device)

# Create output directories for each class
for class_label in minority_data['dx'].unique():
    os.makedirs(os.path.join(output_dir, class_label), exist_ok=True)

class_to_idx = {label: idx for idx, label in enumerate(minority_data['dx'].unique())}

# Loop through each class and generate images
for class_label, count in class_counts.items():
    n_to_generate = 6705 - count  # Calculate how many synthetic images you need
    class_folder = os.path.join(output_dir, class_label)
    class_idx = class_to_idx[class_label]  # Get the integer label for the class

    for i in range(n_to_generate):
        # Ensure each latent vector is truly random
        z = torch.randn(batch_size, latent_dim).to(device)  # Random latent vector for batch size 1

        # Create a tensor for the label (class index)
        labels = torch.tensor([class_idx] * batch_size).to(device)

        # Generate a synthetic image using the GAN generator
        with torch.no_grad():
            synthetic_image = generator(z, labels)  # Pass both latent vector and label

        # Rescale the image from [-1, 1] to [0, 1]
        synthetic_image = 0.5 * (synthetic_image + 1)

        # Save the image in the class-specific folder
        save_image(synthetic_image, os.path.join(class_folder, f'synthetic_{i}.png'))
